In [2]:
import os
os.chdir('/home/sagemaker-user/ml-ops/lab3')
os.listdir()

['.env-sample',
 '.gitignore',
 'Dockerfile',
 'Dockerfile.airflow',
 'Dockerfile.mlflow',
 'Dockerfile.ms',
 'Dockerfile.train',
 'README.md',
 'dags',
 'data',
 'docker-compose.yaml',
 'requirements-airflow.txt',
 'requirements.txt',
 'scripts',
 'src',
 'sm']

In [3]:
import importlib
import src.sm.processing.data_prep as dp

importlib.reload(dp)

dv = dp.build_training_dataset(data_version="debug")
dv

>>> DATA VERSION: debug
>>> Loading GOLDEN train from: s3://mlops-project-sm/data/raw/historical/train.csv
    GOLDEN train shape: (10003, 2)
>>> Loading GOLDEN test  from: s3://mlops-project-sm/data/raw/historical/test.csv
    GOLDEN test shape:  (3080, 2)
>>> FEEDBACK inference path:  s3://mlops-project-sm/data/raw/logs/inference/*/*.jsonl
>>> FEEDBACK corrections path:s3://mlops-project-sm/data/raw/logs/corrections/*/*.jsonl
>>> Globbing files: s3://mlops-project-sm/data/raw/logs/inference/*/*.jsonl
    Found 11 files
    Reading mlops-project-sm/data/raw/logs/inference/2025-12-19/0cc55f39-ff12-47ce-bac3-cdb4a0f81724.jsonl
    Reading mlops-project-sm/data/raw/logs/inference/2025-12-19/2eb6cf43-b138-4f4d-895a-f71f71f56c04.jsonl
    Reading mlops-project-sm/data/raw/logs/inference/2025-12-19/a58be50b-f0e4-4261-a8d1-43668b67c706.jsonl
    Reading mlops-project-sm/data/raw/logs/inference/2025-12-19/ae315ca7-b6b7-4a24-a59b-303e29a20696.jsonl
    Reading mlops-project-sm/data/raw/logs/in

'debug'

In [ ]:
import os
import sagemaker
from datetime import datetime, UTC
import uuid
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

def generate_run_id(prefix: str = "run") -> str:
    """Generate a unique run_id using UTC timestamp + short UUID."""
    timestamp = datetime.now(UTC).strftime("%Y-%m-%d-%H-%M-%S")
    short_uuid = uuid.uuid4().hex[:6]
    return f"{prefix}-{timestamp}-{short_uuid}"

# SageMaker session and execution role
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

# Explicit project bucket (no default bucket)
BUCKET = "mlops-project-sm"

# Project S3 root prefix (your objects live under s3://bucket/data/...)
S3_ROOT = "data"
RAW_PREFIX = f"{S3_ROOT}/raw"
PROCESSED_PREFIX = f"{S3_ROOT}/processed"

# Generate run_id for this processing run
run_id = generate_run_id("banking-prep")
print("Using run_id:", run_id)

# Input must point to a NON-empty S3 prefix
raw_input_s3 = f"s3://{BUCKET}/{RAW_PREFIX}/historical/"

# Output for this run_id
processed_output_s3 = f"s3://{BUCKET}/{PROCESSED_PREFIX}/runs/{run_id}/"

# Absolute path to your processing script
script_path = os.path.abspath("src/sm/processing/data_prep.py")
print("Script path:", script_path)

processor = SKLearnProcessor(
    framework_version="1.2-1",
    role=role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name="banking-data-prep",
    sagemaker_session=sess,
)

processor.run(
    code=script_path,
    arguments=[
        "--data_version", "auto",
        "--input_dir", "/opt/ml/processing/input",
        "--output_dir", "/opt/ml/processing/output",
        "--run_id", run_id,
    ],
    inputs=[
        ProcessingInput(
            source=raw_input_s3,
            destination="/opt/ml/processing/input",
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output",
            destination=processed_output_s3,
        )
    ],
    logs=True,
)


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


Using run_id: banking-prep-2025-12-25-01-38-29-53084b
Script path: /home/sagemaker-user/ml-ops/lab3/src/sm/processing/data_prep.py


INFO:sagemaker:Creating processing-job with name banking-data-prep-2025-12-25-01-38-29-729


.

In [10]:
import sagemaker
sess = sagemaker.Session()
print("SageMaker region:", sess.boto_region_name)

SageMaker region: us-east-1


In [13]:
!aws s3api get-bucket-location --bucket mlops-project-sm

{
    "LocationConstraint": null
}


In [15]:
!aws s3 ls s3://mlops-project-sm/ | head -n 50
!aws s3 ls s3://mlops-project-sm/data/ | head -n 50
!aws s3 ls s3://mlops-project-sm/data/raw/ | head -n 50

                           PRE data/
                           PRE mlflow_experiments/
                           PRE processed/
                           PRE raw/
                           PRE historical/
                           PRE logs/
